In [1]:
%pip install pandas_datareader pykrx yfinance

# src

## common

### path.py

In [1]:
class PATH:
    import os

    try:
        ROOT = os.path.dirname(__file__)
        while not ROOT.endswith('yuho'):
            ROOT = os.path.dirname(ROOT)
    except NameError:
        ROOT = 'https://raw.githubusercontent.com/labwons/yuho/main/'

    try:
        DESKTOP = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
        DOWNLOADS = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Downloads')
    except KeyError:
        DESKTOP = DOWNLOADS = ROOT

    GROUP  = os.path.join(ROOT, r'src/fetch/market/json/group.json')
    INDEX  = os.path.join(ROOT, r'src/fetch/market/json/index.json')
    SPEC  = os.path.join(ROOT, r'src/fetch/market/json/spec.json')
    STATE  = os.path.join(ROOT, r'src/fetch/market/json/state.json')

    BUBBLE = os.path.join(ROOT, r'docs/src/json/bubble.json')
    MAP    = os.path.join(ROOT, r'docs/src/json/treemap.json')
    MACRO  = os.path.join(ROOT, r'docs/src/json/macro.json')



if __name__ == "__main__":
    print(PATH.ROOT)
    print(PATH.GROUP)
    print(PATH.STATE)
    print(PATH.SPEC)
    print(PATH.INDEX)
    print(PATH.MAP)

https://raw.githubusercontent.com/labwons/yuho/main/
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/group.json
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/state.json
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/spec.json
https://raw.githubusercontent.com/labwons/yuho/main/src/fetch/market/json/index.json
https://raw.githubusercontent.com/labwons/yuho/main/docs/src/json/treemap.json


## fetch

### market

#### group.py

In [2]:
# import PATH
from pandas import (
    DataFrame,
    concat,
    Index,
    read_json
)
from pykrx.stock import get_index_portfolio_deposit_file
from re import compile
from requests import get
from requests.exceptions import JSONDecodeError
from time import sleep, time
from typing import (
    Dict,
    Iterable,
    List
)


SECTOR_CODE:Dict[str, str] = {
    'WI100': '에너지', 'WI110': '화학',
    'WI200': '비철금속', 'WI210': '철강', 'WI220': '건설', 'WI230': '기계', 'WI240': '조선', 'WI250': '상사,자본재', 'WI260': '운송',
    'WI300': '자동차', 'WI310': '화장품,의류', 'WI320': '호텔,레저', 'WI330': '미디어,교육', 'WI340': '소매(유통)',
    'WI400': '필수소비재', 'WI410': '건강관리',
    'WI500': '은행', 'WI510': '증권', 'WI520': '보험',
    'WI600': '소프트웨어', 'WI610': 'IT하드웨어', 'WI620': '반도체', 'WI630': 'IT가전', 'WI640': '디스플레이',
    'WI700': '통신서비스',
    'WI800': '유틸리티'
}
CODE_LABEL:Dict[str, str] = {
    'CMP_CD': 'ticker', 'CMP_KOR': 'name',
    'SEC_CD': 'sectorCode', 'SEC_NM_KOR': 'sectorName',
    'IDX_CD': 'industryCode', 'IDX_NM_KOR': 'industryName',
}


class MarketGroup(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        stime = time()
        if not update:
            super().__init__(read_json(PATH.GROUP, orient='index'))
            self.index = self.index.astype(str).str.zfill(6)
            self.index.name = 'ticker'
            return

        date = self.fetchTradingDate()
        self.log = f'Begin [Market Group Fetch] @{date}'
        objs = []
        for n, (code, name) in enumerate(SECTOR_CODE.items()):
            obj = self.fetchWiseGroup(code, date)
            objs.append(obj)
            proc = f"... {n + 1} / {len(SECTOR_CODE)} : {code} {name} :: "
            self.log = f"{proc}Fail" if obj.empty else f"{proc}Success"

        super().__init__(concat(objs, axis=0, ignore_index=True))
        self.drop(columns=[key for key in self if not key in CODE_LABEL], inplace=True)
        self.drop(index=self[self['SEC_CD'].isna()].index, inplace=True)
        self.rename(columns=CODE_LABEL, inplace=True)
        self.set_index(keys="ticker", inplace=True)
        self['industryName'] = self['industryName'].str.replace("WI26 ", "")

        kq, lg = self.fetchKosdaqList(self.index), self.fetchLargeCapList(self.index)
        self.loc[kq, 'name'] = self.loc[kq, 'name'] + '*'
        self.loc[lg, 'stockSize'] = 'large'
        self.log = "... Identify Tickers: Success (N F/S)"

        sc_mdi = self[(self['industryCode'] == 'WI330') & (self['sectorCode'] == 'G50')].index
        sc_edu = self[(self['industryCode'] == 'WI330') & (self['sectorCode'] == 'G25')].index
        sc_sw = self[(self['industryCode'] == 'WI600') & (self['sectorCode'] == 'G50')].index
        sc_it = self[(self['industryCode'] == 'WI600') & (self['sectorCode'] == 'G45')].index
        self.loc[sc_mdi, 'industryCode'], self.loc[sc_mdi, 'industryName'] = 'WI331', '미디어'
        self.loc[sc_edu, 'industryCode'], self.loc[sc_edu, 'industryName'] = 'WI332', '교육'
        self.loc[sc_sw, 'industryCode'], self.loc[sc_sw, 'industryName'] = 'WI601', '소프트웨어'
        self.loc[sc_it, 'industryCode'], self.loc[sc_it, 'industryName'] = 'WI602', 'IT서비스'

        self.log = f'End [Market Group Fetch] / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def fetchTradingDate(cls) -> str:
        URL = 'https://www.wiseindex.com/Index/Index#/G1010.0.Components'
        pattern = compile(r"var\s+dt\s*=\s*'(\d{8})'")
        return pattern.search(get(URL).text).group(1)

    @classmethod
    def fetchWiseGroup(cls, code:str, date:str="", countdown:int=5) -> DataFrame:
        resp = get(f'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={date}&sec_cd={code}')
        try:
            return DataFrame(resp.json()['list'])
        except JSONDecodeError:
            if countdown == 0:
                return DataFrame()
            sleep(5)
            return cls.fetchWiseGroup(code, date, countdown - 1)

    @classmethod
    def fetchKosdaqList(cls, _tickers:Index=None) -> List[str]:
        tickers = get_index_portfolio_deposit_file('2001')
        if not _tickers.empty:
            tickers = [ticker for ticker in _tickers if ticker in tickers]
        return tickers

    @classmethod
    def fetchLargeCapList(cls, _tickers:Index=None) -> List[str]:
        tickers = get_index_portfolio_deposit_file('2203') \
                + get_index_portfolio_deposit_file('1028')
        if not _tickers.empty:
            tickers = [ticker for ticker in _tickers if ticker in tickers]
        return tickers


if __name__ == "__main__":
    marketGroup = MarketGroup(True)
    # print(marketGroup)
    print(marketGroup.log)


Begin [Market Group Fetch] @20250205
... 1 / 26 : WI100 에너지 :: Success
... 2 / 26 : WI110 화학 :: Success
... 3 / 26 : WI200 비철금속 :: Success
... 4 / 26 : WI210 철강 :: Success
... 5 / 26 : WI220 건설 :: Success
... 6 / 26 : WI230 기계 :: Success
... 7 / 26 : WI240 조선 :: Success
... 8 / 26 : WI250 상사,자본재 :: Success
... 9 / 26 : WI260 운송 :: Success
... 10 / 26 : WI300 자동차 :: Success
... 11 / 26 : WI310 화장품,의류 :: Success
... 12 / 26 : WI320 호텔,레저 :: Success
... 13 / 26 : WI330 미디어,교육 :: Success
... 14 / 26 : WI340 소매(유통) :: Success
... 15 / 26 : WI400 필수소비재 :: Success
... 16 / 26 : WI410 건강관리 :: Success
... 17 / 26 : WI500 은행 :: Success
... 18 / 26 : WI510 증권 :: Success
... 19 / 26 : WI520 보험 :: Success
... 20 / 26 : WI600 소프트웨어 :: Success
... 21 / 26 : WI610 IT하드웨어 :: Success
... 22 / 26 : WI620 반도체 :: Success
... 23 / 26 : WI630 IT가전 :: Success
... 24 / 26 : WI640 디스플레이 :: Success
... 25 / 26 : WI700 통신서비스 :: Success
... 26 / 26 : WI800 유틸리티 :: Success
... Identify Tickers: Success (N F/S)
End 

#### index.py

In [4]:
# import PATH
from pandas import (
    DataFrame,
    read_json,
    to_datetime
)
from pykrx.stock import (
    get_index_ohlcv_by_date,
    get_nearest_business_day_in_a_week
)
from re import compile, search
from requests import get
from time import sleep, time
from typing import (
    Dict,
    List
)
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


INDEX_CODE:Dict[str, str] = {
    '1001': 'KOSPI', '2001': 'KOSDAQ',
    'WI100': '에너지', 'WI110': '화학',
    'WI200': '비철금속', 'WI210': '철강', 'WI220': '건설', 'WI230': '기계', 'WI240': '조선', 'WI250': '상사,자본재', 'WI260': '운송',
    'WI300': '자동차', 'WI310': '화장품,의류', 'WI320': '호텔,레저', 'WI330': '미디어,교육', 'WI340': '소매(유통)',
    'WI400': '필수소비재', 'WI410': '건강관리',
    'WI500': '은행', 'WI510': '증권', 'WI520': '보험',
    'WI600': '소프트웨어', 'WI610': 'IT하드웨어', 'WI620': '반도체', 'WI630': 'IT가전', 'WI640': '디스플레이',
    'WI700': '통신서비스',
    'WI800': '유틸리티'
}


class MarketIndex(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        stime = time()
        super().__init__(read_json(PATH.INDEX, orient='index'))
        self.index = self.index.date
        if not update:
            return

        trading_date = get_nearest_business_day_in_a_week()
        server_date = self.fetchServerDate()
        self.log = f'Begin [Market Index Fetch] @{trading_date}'

        for n, (code, name) in enumerate(INDEX_CODE.items()):
            proc = f'... ({n + 1} / {len(INDEX_CODE)}) : {code} {name} :: '
            start = self[code].dropna().index[-1]
            end = trading_date if code in ['1001', '2001'] else server_date
            if start == end:
                continue
            fetch = self.fetchWiseSeries(code, f'{start}', f'{end}')
            if fetch.empty:
                self.log = f'{proc}Fail'
                continue
            for dt in fetch.index:
                self.at[dt, code] = fetch.loc[dt, code]
            self.log = f'{proc}Success '
        self.log = f'End [Market Index Fetch] / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def _netDate2normDate(cls, timestamp:str):
        timestamp = int(search(r'\((\d+)\)', timestamp).group(1))
        return to_datetime(timestamp, unit='ms', utc=True) \
               .tz_convert('Asia/Seoul') \
               .date()

    @classmethod
    def fetchServerDate(cls) -> str:
        URL = 'https://www.wiseindex.com/Index/Index#/G1010.0.Components'
        pattern = compile(r"var\s+dt\s*=\s*'(\d{8})'")
        return pattern.search(get(URL).text).group(1)

    @classmethod
    def fetchWiseSeries(cls, code:str, start:str, end:str, countdown:int=5) -> DataFrame:
        if code in ['1001', '2001']:
            fetch = get_index_ohlcv_by_date(start, end, code, 'd', False)
            fetch.index = fetch.index.date
            fetch = fetch.rename(columns={"종가": code})
            return fetch

        resp = get(f'http://www.wiseindex.com/DataCenter/GridData?currentPage=1&endDT={end}&fromDT={start}&index_ids={code}&isEnd=1&itemType=1&perPage=10000&term=1')
        try:
            fetch = DataFrame(resp.json())[["TRD_DT", "IDX1_VAL1"]]
            fetch["TRD_DT"] = fetch["TRD_DT"].apply(cls._netDate2normDate)
            return fetch.rename(columns={"IDX1_VAL1": code}).set_index(keys="TRD_DT")
        except JSONDecodeError:
            if countdown == 0:
                return DataFrame()
            sleep(5)
            return cls.fetchWiseSeries(code, start, end, countdown - 1)


if __name__ == "__main__":
    marketIndex = MarketIndex(True)
    # print(marketIndex)
    print(marketIndex.log)


Begin [Market Index Fetch] @20250206
... (1 / 28) : 1001 KOSPI :: Success 
... (2 / 28) : 2001 KOSDAQ :: Success 
... (3 / 28) : WI100 에너지 :: Success 
... (4 / 28) : WI110 화학 :: Success 
... (5 / 28) : WI200 비철금속 :: Success 
... (6 / 28) : WI210 철강 :: Success 
... (7 / 28) : WI220 건설 :: Success 
... (8 / 28) : WI230 기계 :: Success 
... (9 / 28) : WI240 조선 :: Success 
... (10 / 28) : WI250 상사,자본재 :: Success 
... (11 / 28) : WI260 운송 :: Success 
... (12 / 28) : WI300 자동차 :: Success 
... (13 / 28) : WI310 화장품,의류 :: Success 
... (14 / 28) : WI320 호텔,레저 :: Success 
... (15 / 28) : WI330 미디어,교육 :: Success 
... (16 / 28) : WI340 소매(유통) :: Success 
... (17 / 28) : WI400 필수소비재 :: Success 
... (18 / 28) : WI410 건강관리 :: Success 
... (19 / 28) : WI500 은행 :: Success 
... (20 / 28) : WI510 증권 :: Success 
... (21 / 28) : WI520 보험 :: Success 
... (22 / 28) : WI600 소프트웨어 :: Success 
... (23 / 28) : WI610 IT하드웨어 :: Success 
... (24 / 28) : WI620 반도체 :: Success 
... (25 / 28) : WI630 IT가전 :: Success 
... 

#### state.py

In [ ]:
# import PATH
from datetime import datetime, timedelta
from pandas import (
    concat,
    DataFrame,
    Index,
    read_json
)
from pykrx.stock import (
    get_exhaustion_rates_of_foreign_investment,
    get_nearest_business_day_in_a_week,
    get_market_cap_by_ticker,
    get_market_fundamental,
    get_market_ohlcv_by_date
)
from requests.exceptions import JSONDecodeError, SSLError
from time import time


CAP_LABEL:Dict[str, str] = {
    '종가':'close', '시가총액':'marketCap',
    '거래량':'volume', '거래대금':'amount', '상장주식수':'shares'
}
MUL_LABEL:Dict[str, str] = {
    'PER': 'PER(Q)', 'PBR': 'PBR(Q)', 'DIV': 'DIV(Q)'
}
PCT_LABEL:Dict[str, str] = {"지분율":'foreignRate'}
PRC_LABEL:Dict[str, str] = {
    "시가":"open", "고가":"high", "저가":"low", "종가":"close",
    "거래량":"volume", "거래대금":"amount"
}
INTERVALS:Dict[str, int] = {
    'D+0': 0, 'D-1': 1, 'W-1': 7,
    'M-1': 30, 'M-3': 91, 'M-6': 182, 'Y-1': 365
}


class MarketState(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        stime = time()
        if not update:
            super().__init__(read_json(PATH.STATE, orient='index'))
            return

        objs = []
        date = get_nearest_business_day_in_a_week()
        self.log = f'Begin [Market State Fetch] @{date}'

        marketCap = self.fetchMarketCap(date)
        objs.append(marketCap)
        self.log = f'... Fetch Market Cap :: {"Fail" if marketCap.empty else "Success"}'

        multiples = self.fetchMultiples(date)
        objs.append(multiples)
        self.log = f'... Fetch Multiples :: {"Fail" if multiples.empty else "Success"}'

        foreignRate = self.fetchForeignRate(date)
        objs.append(foreignRate)
        self.log = f'... Fetch Foreign Rate :: {"Fail" if foreignRate.empty else "Success"}'
        subset = concat(objs, axis=1)

        returns = self.fetchReturns(date)
        self.log = f'... Fetch Returns :: {"Fail" if returns.empty else "Success"}'

        merge = returns.join(subset, how='left')
        merge = merge[~merge.index.isin(self.fetchKonexList(date))]
        super().__init__(merge)

        self.log = f'End [Market State Fetch] / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def fetchMarketCap(cls, date:str) -> DataFrame:
        try:
            df = get_market_cap_by_ticker(date=date, market='ALL', alternative=True) \
                 .rename(columns=CAP_LABEL)
            df.index.name = 'ticker'
            return df
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(CAP_LABEL.values()))

    @classmethod
    def fetchKonexList(cls, date:str) -> Index:
        try:
            return get_market_cap_by_ticker(date=date, market='KONEX').index
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return []

    @classmethod
    def fetchMultiples(cls, date:str) -> DataFrame:
        try:
            df = get_market_fundamental(date=date, market="ALL") \
                 .rename(columns=MUL_LABEL)
            df.index.name = "ticker"
            return df[MUL_LABEL.values()]
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=MUL_LABEL.values())

    @classmethod
    def fetchForeignRate(cls, date:str) -> DataFrame:
        try:
            df = get_exhaustion_rates_of_foreign_investment(date=date, market='ALL') \
                 .rename(columns=PCT_LABEL)
            df.index.name = 'ticker'
            return round(df[PCT_LABEL.values()].astype(float), 2)
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=PCT_LABEL.values())

    @classmethod
    def fetchReturns(cls, date:str, tickers:Iterable=None) -> DataFrame:
        tdate = datetime.strptime(date, "%Y%m%d")
        intv = {key: tdate - timedelta(val) for key, val in INTERVALS.items()}
        objs = {
            key: cls.fetchMarketCap(val.strftime("%Y%m%d"))
            for key, val in intv.items()
        }
        base = concat(objs, axis=1)
        base = base[
            (~base['D+0']['shares'].isna()) &
            (base['D+0']['marketCap'] >= base['D+0']['marketCap'].median())
        ]

        returns = concat({
            dt: base[dt]['close'] / base['D+0']['close'] - 1 for dt in objs
        }, axis=1)
        returns.drop(columns=['D+0'], inplace=True)

        diff = base[base['Y-1']['shares'] != base['D+0']['shares']].index
        fdate = (tdate - timedelta(380)).strftime("%Y%m%d")
        for ticker in diff:
            ohlc = get_market_ohlcv_by_date(fromdate=fdate, todate=date, ticker=ticker)
            for interval in returns.columns:
                ohlc_copy = ohlc[ohlc.index >= intv[interval]]['종가']
                returns.loc[ticker, interval] = ohlc_copy.iloc[-1] / ohlc_copy.iloc[0] - 1
        return round(100 * returns, 2)


if __name__ == "__main__":
    marketState = MarketState(True)
    print(marketState)
    print(marketState.log)



#### spec.py

In [5]:
# import PATH
from datetime import datetime
from numpy import nan, isnan
from pandas import (
    concat,
    DataFrame,
    Series
)
from pykrx.stock import get_market_cap_by_ticker
from re import DOTALL, sub
from requests import get
from time import time
from typing import Dict, Union, Tuple
from xml.etree.ElementTree import ElementTree, fromstring


OVERVIEW_TAG:Dict[str, str] = {
    # 'date': 'price/date',
    'high52': 'price/high52week',
    'low52': 'price/low52week',
    'beta': 'price/beta',
    'floatShares': 'price/ff_sher_rt',
    'estPrice': 'consensus/target_price',
    'estEps': 'consensus/eps'
}
STATEMENT_TAG:Dict[str, str] = {
    'consolidateAnnual': f'financial_highlight_ifrs_D/financial_highlight_annual',
    'consolidateQuarter': f'financial_highlight_ifrs_D/financial_highlight_quarter',
    'separateAnnual': f'financial_highlight_ifrs_B/financial_highlight_annual',
    'separateQuarter': f'financial_highlight_ifrs_B/financial_highlight_quarter'
}


class MarketSpec(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        stime = time()
        if not update:
            super().__init__(read_json(PATH.SPEC, orient='index'))
            return

        date = datetime.today().strftime("%Y%m%d")
        self.log = f'Begin [Market Spec Fetch] @{date}'

        base = self.fetchMarketCap(date)
        base = base[
            (~base['shares'].isna()) &
            (base['marketCap'] >= base['marketCap'].median())
        ]

        objs = []
        for n, ticker in enumerate(base.index):
            try:
                xml = self.fetchXml(ticker)
                obj = self.fetchOverview(xml)
                A, Q = self.fetchStatement(xml, False)

                obj['trailingSales'] = Q[Q.columns[0]].sum()
                obj['trailingEarning'] = Q['영업이익(억원)'].sum()
                obj['trailingNetIncome'] = quarter['당기순이익(억원)'].sum()
                obj['trailingEarningRatio'] = round(quarter['영업이익률(%)'].mean(), 2)
                obj['trailingEps'] = compensationSum(quarter['EPS(원)'])

                obj.name = ticker
                objs.append(obj)
            except:
                self.log = f'... Fetch {ticker} :: Fail'
                continue

        super().__init__(concat(objs, axis=1).T)
        self.log = f'End [Market Spec Fetch] / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def _format(cls, num:str) -> Union[int, float]:
        if not num:
            return nan
        try:
            return float(num) if "." in num else int(num)
        except ValueError:
            num = "".join([c for c in num if c.isdigit() or c in [".", "-"]])
            if not num or num == "." or num == "-":
                return nan
            return float(num) if "." in num else int(num)

    @classmethod
    def fetchMarketCap(cls, date:str) -> DataFrame:
        try:
            df = get_market_cap_by_ticker(date=date, market='ALL', alternative=True) \
                 .rename(columns=CAP_LABEL)
            df.index.name = 'ticker'
            return df
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(CAP_LABEL.values()))

    @classmethod
    def fetchXml(cls, ticker:str) -> ElementTree:
        resp = get(url=f"http://cdn.fnguide.com/SVO2/xml/Snapshot_all/{ticker}.xml")
        resp.encoding = 'euc-kr'
        text = resp.text.replace("<![CDATA[", "").replace("]]>", "")
        text = sub(r'<business_summary>.*?</business_summary>', '', text, flags=DOTALL)
        return fromstring(text)

    @classmethod
    def fetchOverview(cls, xml:ElementTree) -> Series:
        data = {}
        for key, tag in OVERVIEW_TAG.items():
            ftag = xml.find(tag)
            data[key] = nan if ftag is None else ftag.text
        return Series(data=data).apply(cls._format)

    @classmethod
    def fetchStatement(cls, xml:ElementTree, include_estimated:bool=True) -> Tuple[DataFrame, DataFrame]:
        def _statement(tag:str) -> DataFrame:
            obj = xml.find(tag)
            if obj is None:
                return DataFrame()
            columns = [val.text for val in obj.findall('field')]
            index, data = [], []
            for record in obj.findall('record'):
                index.append(record.find('date').text)
                data.append([val.text for val in record.findall('value')])
            df = DataFrame(index=index, columns=columns, data=data)
            if not include_estimated:
                df = df[(~df.index.str.endswith('(E)')) & (~df.index.str.endswith('(P)'))]
            return df.map(cls._format)

        annual = _statement(STATEMENT_TAG['consolidateAnnual'])
        quarter = _statement(STATEMENT_TAG['consolidateQuarter'])
        if isnan(quarter.iloc[-1]['부채비율(%)']):
            annual = _statement(STATEMENT_TAG['separateAnnual'])
            quarter = _statement(STATEMENT_TAG['separateQuarter'])
        return annual, quarter

if __name__ == "__main__":
    marketSpec = MarketSpec(True)
    print(marketSpec)
    print(marketSpec.log)


nan


Index(['매출액(억원)', '영업이익(억원)', '영업이익(발표기준)', '당기순이익(억원)', '  지배주주순이익(억원)',
       '  비지배주주순이익(억원)', '자산총계(억원)', '부채총계(억원)', '자본총계(억원)', '  지배주주지분(억원)',
       '  비지배주주지분(억원)', '자본금(억원)', '부채비율(%)', '유보율(%)', '영업이익률(%)',
       '지배주주순이익률(%)', 'ROA(%)', 'ROE(%)', 'EPS(원)', 'BPS(원)', 'DPS(원)',
       'PER(배)', 'PBR(배)', '발행주식수(천주)', '배당수익률(%)'],
      dtype='object')

# Experiment